In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [2]:
import numpy as np
import pandas as pd
import os
import json
import transformers
import numpy as np
import pandas as pd
import os
import json
import transformers
import datasets
import torch
import sklearn
import datasets
import evaluate
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [3]:
df = pd.read_csv("file.csv")

In [4]:
df.head()

,topic,source,bias,url,title,date,authors,content,content_original,source_url,bias_text,ID
0,terrorism,New York Times - News,0,http://www.nytimes.com/2016/09/20/nyregion/ahm...,"Bomb Suspect Changed After Trip Abroad, Friend...",2016-09-20,N. R. Kleinfield,"Besides his most recent trip to Quetta , Mr. R...","Besides his most recent trip to Quetta, Mr. Ra...",www.nytimes.com,left,004Gt3gcsotuiYmz
1,supreme_court,Vox,0,https://www.vox.com/policy-and-politics/2018/9...,Why Susan Collins claims she’s being bribed ov...,2018-09-12,"Emily Stewart, Terry Nguyen, Rebecca Jennings,...",Is Maine Republican Sen. Susan Collins being b...,Is Maine Republican Sen. Susan Collins being b...,www.vox.com,left,00eP4XD3VdMmHITE
2,education,Ezra Klein,0,http://www.npr.org/blogs/thetwo-way/2014/05/06...,Poll: Prestigious Colleges Won't Make You Happ...,2014-05-06,Anya Kamenetz,Poll : Prestigious Colleges Wo n't Make You Ha...,Poll: Prestigious Colleges Won't Make You Happ...,www.npr.org,left,00FTGIZEd6B8zQ4U
3,us_house,Breitbart News,2,http://www.breitbart.com/big-government/2017/0...,Paul Ryan Reportedly Says No Chance for Border...,2017-09-12,Ian Mason,"House Speaker Paul Ryan , at a private dinner ...","House Speaker Paul Ryan, at a private dinner e...",www.breitbart.com,right,00HGGqBRf1kzPRlg
4,white_house,Guest Writer - Left,0,https://www.cnn.com/2019/07/11/politics/donald...,OPINION: Trump seeking change of legal fortune...,2019-07-11,Analysis Stephen Collinson,( CNN ) President Donald Trump has reason to h...,(CNN) President Donald Trump has reason to hop...,www.cnn.com,left,00IzI5ynahBVtC9l


In [5]:
print(df.isnull().sum())


print(df['bias_text'].value_counts())

topic                  0
source                 0
bias                   0
url                    0
title                  0
date                4407
authors             9668
content                0
content_original       0
source_url             0
bias_text              0
ID                     0
dtype: int64
bias_text
right     13734
left      13005
center    10815
Name: count, dtype: int64


For Now we do not need to focus on the authors as we are not going to use that column but yes it will play a role in future


In [6]:
#On the research paper from which we have extracted the data ,they already have the classified into training and test data along with validation data we are using that only for our project also .


split_dir = 'Article-Bias-Prediction-main/data/splits/random/'
# the id and bias are seperated by t and not by comma

train_split = pd.read_csv(os.path.join(split_dir, 'train.tsv'), sep = '\t')
val_split = pd.read_csv(os.path.join(split_dir, 'valid.tsv'), sep = '\t')
test_split = pd.read_csv(os.path.join(split_dir, 'test.tsv'), sep = '\t')
# the id and bias are seperated by t and not by comma
print(train_split.columns)

df_train = df[df['ID'].isin(train_split['ID'])]
df_val = df[df['ID'].isin(val_split['ID'])]
df_test = df[df['ID'].isin(test_split['ID'])]

print(f"Train: {len(df_train)}, Validation: {len(df_val)}, Test: {len(df_test)}")

Index(['ID', 'bias'], dtype='object')
Train: 27978, Validation: 6996, Test: 1300


In [7]:
df_train.loc[: , 'content_length'] = df_train['content'].apply(lambda x: len(str(x).split()))
print(df_train[ 'content_length'].describe())

count    27978.000000
mean      1084.579777
std        741.086921
min         26.000000
25%        623.000000
50%        922.000000
75%       1304.000000
max       9703.000000
Name: content_length, dtype: float64


C:\Users\shubh\AppData\Local\Temp\ipykernel_7372\2680055546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[: , 'content_length'] = df_train['content'].apply(lambda x: len(str(x).split()))


In [ ]:
#Seeing the data types
print(df_train['content'].apply(type).value_counts())

content
<class 'str'>    27978
Name: count, dtype: int64


In [ ]:
#checking for null values
print(df_train['content'].isnull().sum())
print((df_train['content'].str.strip() == '').sum())
print(df_train['bias'].isnull().sum())


0
0
0


In [ ]:
#if there is by chance any value left with no bias value then it gets filled with neutral
df_train.loc[df_train['bias'].isnull(), 'bias'] = 1

print("Missing value filled with value 1 (neutral Bias)")

Missing value filled with value 1 (neutral Bias)


In [ ]:
#vectorization 

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(df_train['content']).toarray()
X_val = tfidf.transform(df_val['content']).toarray()
X_test = tfidf.transform(df_test['content']).toarray()

print("TF-IDF Vectorization successful.")
print(f"Train set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")


TF-IDF Vectorization successful.
Train set shape: (27978, 5000)
Validation set shape: (6996, 5000)
Test set shape: (1300, 5000)


In [12]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# model = LogisticRegression(max_iter=500, random_state=42)
# model.fit(X_train, df_train['bias'])


# y_pred_val = model.predict(X_val)

# print(f"Validation Accuracy: {accuracy_score(df_val['bias'], y_pred_val):.4f}")
# print("\nValidation Classification Report:\n", classification_report(df_val['bias'], y_pred_val))


In [13]:
# from sklearn.ensemble import RandomForestClassifier

# # Initialize Random Forest
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train, df_train['bias'])

# # Evaluate on Validation Set
# y_pred_rf = rf_model.predict(X_val)
# print(f"Random Forest Validation Accuracy: {accuracy_score(df_val['bias'], y_pred_rf):.4f}")
# print("\nRandom Forest Classification Report:\n", classification_report(df_val['bias'], y_pred_rf))


In [14]:
# from xgboost import XGBClassifier

# # Initialize XGBoost
# xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
# xgb_model.fit(X_train, df_train['bias'])

# # Evaluate on Validation Set
# y_pred_xgb = xgb_model.predict(X_val)
# print(f"XGBoost Validation Accuracy: {accuracy_score(df_val['bias'], y_pred_xgb):.4f}")
# print("\nXGBoost Classification Report:\n", classification_report(df_val['bias'], y_pred_xgb))


In [ ]:
# map the labels
label_mapping = {'left': 0, 'center': 1, 'right': 2}
df_train.loc[:,'bias'] = df_train['bias'].replace(label_mapping)
df_val.loc[:,'bias'] = df_val['bias'].replace(label_mapping)
df_test.loc[:,'bias'] = df_test['bias'].replace(label_mapping)


df_combined = pd.concat([df_train, df_val, df_test])


dataset = Dataset.from_pandas(df_combined[['content', 'bias']])
dataset = dataset.shuffle(seed=42)
print(dataset)


Dataset({
    features: ['content', 'bias', '__index_level_0__'],
    num_rows: 36274
})


In [ ]:
# tokenize the data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples['content'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/36274 [00:00<?, ? examples/s]

In [ ]:
#split the data for train and testing randomly
train_testvalid = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

train_data = train_testvalid['train']
val_data = test_valid['train']
test_data = test_valid['test']

print(f"Training Size: {len(train_data)}")
print(f"Validation Size: {len(val_data)}")
print(f"Test Size: {len(test_data)}")


Training Size: 29019
Validation Size: 3627
Test Size: 3628


In [18]:
def format_dataset(example):
    return {'labels': example['bias'], **example}

# Format datasets first
train_data = train_data.map(format_dataset)
val_data = val_data.map(format_dataset)
test_data = test_data.map(format_dataset)  # Optional unless used

Map:   0%|          | 0/29019 [00:00<?, ? examples/s]

Map:   0%|          | 0/3627 [00:00<?, ? examples/s]

Map:   0%|          | 0/3628 [00:00<?, ? examples/s]

In [ ]:
# ensure it runs on gpu 

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Trainer will use:", trainer.args.device)

print(torch.cuda.is_available())           # Should be True
print(torch.cuda.get_device_name(0))       # Should return GPU name
print(torch.cuda.current_device())

Using device: cuda
Trainer will use: cuda:0
True
NVIDIA GeForce RTX 4060 Laptop GPU
0


In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# training arguements
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# metric computation
def compute_metrics(pred):
    metric = evaluate.load("accuracy")
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.485700,0.481527,0.821891
2,0.383300,0.452376,0.846705
3,0.271300,0.587151,0.844775


TrainOutput(global_step=10884, training_loss=0.43732432695225465, metrics={'train_runtime': 5076.2916, 'train_samples_per_second': 17.15, 'train_steps_per_second': 2.144, 'total_flos': 2.2905864807146496e+16, 'train_loss': 0.43732432695225465, 'epoch': 3.0})

In [24]:
results = trainer.evaluate(test_data)
print(results)


{'eval_loss': 0.5806844234466553, 'eval_accuracy': 0.8539140022050716, 'eval_runtime': 55.8948, 'eval_samples_per_second': 64.908, 'eval_steps_per_second': 8.122, 'epoch': 3.0}


In [25]:
model.to('cpu')  #
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.txt',
 './saved_model\\added_tokens.json',
 './saved_model\\tokenizer.json')